In [1]:
# !pip install update pandas

In [2]:
# !pip install scikit-learn konlpy pyLDAvis

In [3]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.lda_model
import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Okt


In [4]:
from google.colab import files
import os

data_dir = 'data'

if not os.path.exists(data_dir):
    os.mkdir(data_dir)
os.chdir(data_dir)
files.upload()
os.chdir('..')

Saving news.csv to news.csv


In [6]:
okt = Okt()
df = pd.read_csv('./data/news.csv')


print(f"Loaded {len(df)} news articles.")

Loaded 1600 news articles.


In [7]:
# 한국어 전처리 함수
def preprocess_korean(text):
    if pd.isna(text):
        return ""
    # 기본: 특수문자, 숫자, 영어 제거 (형태소 분석 없이)
    text = re.sub(r'[^가-힣\s]', ' ', text)  # 한글과 공백만 유지
    text = re.sub(r'\s+', ' ', text).strip()  # 다중 공백 제거

    # 형태소 분석: 명사 추출
    tokens = okt.nouns(text)
    # 불용어 제거 (간단한 한국어 불용어 리스트)
    stopwords_kr = {'것', '수', '이', '가', '를', '을', '에', '의', '와', '과', '로', '으로', '에서', '이다', '되다', '있다', '하다', '않다'}
    tokens = [word for word in tokens if len(word) > 1 and word not in stopwords_kr]
    return ' '.join(tokens)

# 데이터 전처리
df['processed_news'] = df['news'].apply(preprocess_korean)
processed_data = df['processed_news'] #.tolist()

In [8]:
# LDA 모델 학습 (토픽 수: 5로 가정, 조정 가능)
n_topics = 6

# TF-IDF 벡터라이저 (한국어 지원, max_features 제한으로 효율화)
vectorizer = TfidfVectorizer(max_features=1000, min_df=2, max_df=0.8)
feat_vect = vectorizer.fit_transform(processed_data)


lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(feat_vect)

feature_names = vectorizer.get_feature_names_out()

# 각 뉴스 기사의 토픽 분포 및 주요 토픽 할당 (분류)
doc_topics = lda.transform(feat_vect)


# pyLDAvis 시각화
vis = pyLDAvis.lda_model.prepare(lda, feat_vect, vectorizer, mds='tsne')
pyLDAvis.display(vis)
# pyLDAvis.save_html(vis, 'news_lda_visualization.html')

# # 전체 결과 저장 (CSV로)
# df.to_csv('classified_news.csv', index=False)